Architecture of a FHIR app:
![image](https://cdn-images-1.medium.com/max/1200/1*qRd-H-cPHAGTB993sNLy3Q.png)
(from https://blog.heliossoftware.com/fhir-architectural-patterns-ae828b13d40c)

## Initialize the environment to run the application

In [ ]:
# Import the client library that understands how to make FHIR calls and 
# interpret the results
#  Then print 'FHIR client library has been loaded'
from fhirclient import client
print('FHIR client library has been loaded')

## Find a patient with relevant variables

In [ ]:
import requests

In [ ]:
FHIRJSONMimeType = 'application/fhir+json'
header_defaults = {
            'Accept': FHIRJSONMimeType,
            'Accept-Charset': 'UTF-8',
        }

*Hint: Use the URL https://hapi.fhir.org/baseR4/[resource]?code=[loinc]*

Here we will use patient 431798

In [ ]:
patientId = "21f4e9f6-ec3c-405d-97c3-4e69d1534af2"

## Load the data and get basic demographics

In [ ]:
r = requests.get('https://stu3.test.pyrohealth.net/fhir/Patient/21f4e9f6-ec3c-405d-97c3-4e69d1534af2',
                 headers = header_defaults)

In [ ]:
body = r.json()

In [ ]:
type(body)

In [ ]:
# Load the Patient from the database, and display demographics
#   Define the FHIR Endpoint. We define the name of this app (used in SMART 
#   calls), and the location of the server
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://stu3.test.pyrohealth.net/fhir'
}



In [ ]:
# Create an instance of the FHIR client that points to this FHIR server. We use 
# this to make the calls.
db = client.FHIRClient(settings=settings)

In [ ]:
# Perform a GET (read) on the patient with the ID you identified earlier
from fhirclient.models.patient import Patient
patient = Patient.read(patientId, db.server)

In [ ]:
patient.as_json()

## *Optional: get basic demographics*

In [ ]:
# Patient name. 
# Show the name object so we can see what's in it
print(patient.name)

The patient's name is a list of `HumanName`s. We can find out about the structure of this data type on the FHIR website: https://www.hl7.org/fhir/datatypes.html#HumanName

In [ ]:
# Show the contents of each object in the list
for name in patient.name:
    print(name.as_json())

In [ ]:
# We see that a Patient can have more than one name (i.e. Patient.name is a list)
# and each name can have more than one given name (i.e. given is also a list).
# Let's use the first name in the list and the first given name:
first_name = patient.name[0].given[0]
last_name = patient.name[0].family
print(first_name, last_name)

In [ ]:
# Patient's gender
gender = patient.gender
gender

In [ ]:
# Define variable dob as the patient's date of birth
dob = patient.birthDate.date
print(dob)

In [ ]:
# We'd like to know the patient's age. The EHR contains the date of birth only, 
# so we have to calculate the age using the dob and today's date.
# For this we need the datetime library
import datetime

# Define and print today variable.
today = datetime.date.today()
print(today)

In [ ]:
# Calculate the patient's age using the relativedelta method of the dateutil 
# module
from dateutil.relativedelta import relativedelta

delta = relativedelta(today, dob)
age = delta.years
age

In [ ]:
# Print patient name, gender, DOB, age, and today's date
print("Patient's Name(s) =", first_name, last_name)
print("Gender =", gender)
print("DOB =", dob)
print("Today's Date =", today)
print("Patient's age =", age)